In [ ]:
pip install pandas geopandas pyproj

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform

def convert_xls_to_shp(input_folder, output_folder, source_epsg, target_epsg):
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each .xls file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".xls"):
            file_path = os.path.join(input_folder, filename)

            # Read Excel file into a Pandas DataFrame
            df = pd.read_excel(file_path)

            # Create a GeoDataFrame with Point geometries
            geometry = [Point(x, y) for x, y in zip(df['Easting'], df['Northing'])]
            crs_source = {'init': f'epsg:{source_epsg}'}  # Source CRS
            crs_target = {'init': f'epsg:{target_epsg}'}  # Target CRS
            
            gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs_source)

            # Perform coordinate transformation if needed
            if source_epsg != target_epsg:
                project = Proj(crs_target)
                gdf['geometry'] = gdf['geometry'].to_crs(crs_target)

            # Save GeoDataFrame to a Shapefile
            output_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.shp")
            gdf.to_file(output_file)

            print(f"Conversion completed: {output_file}")

# Prompt user for input and output folders
input_folder = input("Enter the input folder path: ")
output_folder = input("Enter the output folder path: ")
source_epsg = input("Enter the source UTM Zone EPSG code: ")
target_epsg = input("Enter the target UTM Zone EPSG code: ")

convert_xls_to_shp(input_folder, output_folder, source_epsg, target_epsg)
